In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.optimize as opt

rf_rate = 0.02

In [2]:
# data: 
xls_dict  = pd.read_excel('data/trading-game-data-08112023.xlsx', sheet_name=None)
info_df = xls_dict['info'][['Ticker', 'RBICS Economy']]
info_df = info_df.set_index('Ticker')

index_price_df = xls_dict['index-price']
price_df = xls_dict['price']
size_df = xls_dict['size']
price_to_book_df = xls_dict['price-to-book']
turnover_df = xls_dict['turnover']

## ESG ratings
We will be using the esg ratings from https://www.gigasheet.com/sample-data/sp-500-esg-risk-ratings. Some companies are present in the dataframe therefore we will be using the sector median (and merged based on the info tab in the Trading game datasheet).

In [3]:
esgratings_df = pd.read_csv('data\esg data.csv')
esgratings_df = esgratings_df[['Symbol', 'Sector', 'Total ESG Risk score']]
sector_medians = esgratings_df.groupby(esgratings_df['Sector']).median()

esgratings_df.set_index('Symbol', inplace=True)
esgratings_df = esgratings_df.T

full_esg_df = pd.DataFrame(index=info_df.index)
full_esg_df['ESG Score'] = np.nan  # Initialize all scores as NaN

# Iterate over all companies in the S&P 500
for company in full_esg_df.index:
    if company in esgratings_df.index:
        # Use the actual ESG score if available
        full_esg_df.loc[company, 'ESG Score'] = esgratings_df.loc[company, 'Total ESG Risk score']
    else:
        # Retrieve the sector from info_df
        sector = info_df.loc[company, 'RBICS Economy']
        # Use the sector median if the sector is available
        if sector in sector_medians.index:
            full_esg_df.loc[company, 'ESG Score'] = sector_medians.loc[sector, 'Total ESG Risk score']

# Handling cases where the sector is unknown or no median is available
full_esg_df['ESG Score'].fillna(full_esg_df['ESG Score'].median(), inplace=True)
full_esg_df['Inverted_ESG'] = -full_esg_df['ESG Score']

In [13]:
def calc_averagereturns():
    price_df = xls_dict['price'].reset_index()
    price_df = price_df.drop('index', axis = 1)
    price_df['Date'] = pd.to_datetime(price_df['Date'])
    price_df.set_index('Date', inplace=True)
    daily_returns = price_df.pct_change()
    daily_returns = daily_returns.drop(daily_returns.index[0])
    expected_returns = daily_returns.mean()
    risk = daily_returns.std()
    average_returns = daily_returns.mean() * len(daily_returns)
    return average_returns, daily_returns

average_returns, daily_returns = calc_averagereturns()


A      -0.274701
AAL    -0.024674
AAPL    0.355328
ABBV   -0.110380
ABNB    0.427907
          ...   
YUM    -0.009085
ZBH    -0.165946
ZBRA   -0.134531
ZION   -0.178025
ZTS     0.159445
Length: 500, dtype: float64

In [15]:
#Calculating the factors per date, SMB
#SMB needs average returns of stocks

def calc_ff_smb():
    size_df = xls_dict['size'].reset_index()
    size_df = size_df.drop('index', axis = 1)
    size_df['Date'] = pd.to_datetime(size_df['Date'])
    size_df.set_index('Date', inplace=True)


    average_returns_top =[]
    average_returns_low = []

    for i in range(len(size_df)-1):
        top_stocks = size_df.iloc[i].nlargest(int(len(size_df.iloc[i])*0.3)).index
        low_stocks = size_df.iloc[i].nsmallest(int(len(size_df.iloc[i])*0.3)).index
        average_returns_top.append(daily_returns.iloc[i][top_stocks].mean())
        average_returns_low.append(daily_returns.iloc[i][low_stocks].mean())

    ave_return_df = pd.DataFrame({'top': average_returns_top, 'low': average_returns_low})
    smb = ave_return_df['low'] - ave_return_df['top']

    return smb

smb = calc_ff_smb()
smb

0      0.000175
1      0.012152
2      0.000187
3      0.000964
4      0.006410
         ...   
209   -0.011296
210    0.006039
211    0.018426
212   -0.011327
213   -0.000621
Length: 214, dtype: float64

In [7]:
# Now HML
#calculate market to book. As price to book / price = 1/book value bepalen
#make new dataframe with price divided by price to book
price_to_book_df = xls_dict['price-to-book'].reset_index()
price_to_book_df = price_to_book_df.drop('index', axis = 1)
price_to_book_df['Date'] = pd.to_datetime(price_to_book_df['Date'])
price_to_book_df.set_index('Date', inplace=True)

bookvalue_df = price_df / price_to_book_df
btm_df = bookvalue_df / size_df
btm_df

def top_low_stocks(row):
    top_stocks = row.nlargest(int(len(row)*0.3)).index
    low_stocks = row.nsmallest(int(len(row)*0.3)).index
    average_returns_top = average_returns[top_stocks].mean()
    average_returns_low = average_returns[low_stocks].mean()
    #print(average_returns_low)

    return average_returns_top, average_returns_low

ave_return_df_hml = btm_df.apply(top_low_stocks, axis = 1, result_type = 'expand')
hml = ave_return_df_hml[1] - ave_return_df_hml[0]
hml
#now Mkt-rf
#Calculate market returns
index_price_df = xls_dict['index-price'].reset_index()
index_price_df = index_price_df.drop('index', axis = 1)
index_price_df['Date'] = pd.to_datetime(index_price_df['Date'])
index_price_df.set_index('Date', inplace=True)
daily_market_returns = index_price_df.pct_change()
mkt_rf = daily_market_returns - rf_rate
mkt_rf

,S&P 500
Date,
2022-12-30,NaN
2023-01-03,-0.024001
2023-01-04,-0.012461
2023-01-05,-0.031646
2023-01-06,0.002841
...,...
2023-11-01,-0.009494
2023-11-02,-0.001141
2023-11-03,-0.010606


In [8]:
#now Mkt-rf
#Calculate risk free rate

#Calculate market returns
index_price_df = xls_dict['index-price'].reset_index()
index_price_df = index_price_df.drop('index', axis = 1)
index_price_df['Date'] = pd.to_datetime(index_price_df['Date'])
index_price_df.set_index('Date', inplace=True)
daily_market_returns = index_price_df.pct_change()
mkt_rf = daily_market_returns - rf_rate
mkt_rf

,S&P 500
Date,
2022-12-30,NaN
2023-01-03,-0.024001
2023-01-04,-0.012461
2023-01-05,-0.031646
2023-01-06,0.002841
...,...
2023-11-01,-0.009494
2023-11-02,-0.001141
2023-11-03,-0.010606


In [9]:
#now RMW
#calculate operating profitability
turnover_df = xls_dict['turnover'].reset_index()
turnover_df = turnover_df.drop('index', axis = 1)
turnover_df['Date'] = pd.to_datetime(turnover_df['Date'])
turnover_df.set_index('Date', inplace=True)



In [10]:
# now CMA
#calculate investment
#make new dataframe with price divided by price to book
price_to_book_df = xls_dict['price-to-book'].reset_index()
price_to_book_df = price_to_book_df.drop('index', axis = 1)
price_to_book_df['Date'] = pd.to_datetime(price_to_book_df['Date'])
price_to_book_df.set_index('Date', inplace=True)



In [ ]:
def calc_normalized_weights(optimal_weights, stock_names, threshold=0.005):
    thresholded_weights = np.where(optimal_weights >= threshold, optimal_weights, 0)

    if np.sum(thresholded_weights) > 0:  # Prevent division by zero
        normalized_weights = thresholded_weights / np.sum(thresholded_weights)
    else:
        normalized_weights = thresholded_weights  # In case all are zero, which should not happen

    # Create a DataFrame from the stock names and their corresponding weights
    df_stocks_with_weights = pd.DataFrame({
        'Stock': stock_names,
        'Weight': normalized_weights
    })

    df_stocks_with_weights = df_stocks_with_weights[df_stocks_with_weights['Weight'] >= threshold]

    return df_stocks_with_weights

stock_names = returns.columns
stocks_with_weights = calc_normalized_weights(optimal_weights, stock_names)
stocks_with_weights

In [2]:
import pandas as pd
from scipy.optimize import minimize
import numpy as np

# Loading your data
significance = pd.read_excel(r'C:\Users\bramg\Desktop\studie\main_studie\files\asset_pricing\assignments_assetpricing\trading_game\data\signif.xlsx')
coefficient = pd.read_excel(r'C:\Users\bramg\Desktop\studie\main_studie\files\asset_pricing\assignments_assetpricing\trading_game\data\params.xlsx')

# Assuming coefficient has one row per stock and columns for different factors or returns
# Ensure coefficient is properly formatted for covariance calculation

def portfolio_risk(weights):
    # Calculate portfolio variance
    # Ensure the dimensions of the covariance matrix match the length of 'weights'
    cov_matrix = coefficient.cov()
    if len(weights) != cov_matrix.shape[0]:
        raise ValueError("Mismatch in dimensions of weights and covariance matrix")
    portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
    return portfolio_variance

# Constraints and bounds
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = tuple((0, 1) for _ in range(len(coefficient)))

# Initial guess (equal distribution)
init_guess = [1/len(coefficient)] * len(coefficient)

# Run the optimizer
optimal_portfolio = minimize(portfolio_risk, init_guess, method='SLSQP', bounds=bounds, constraints=cons)

# Optimal weights
optimal_weights = optimal_portfolio.x
print("Optimal Portfolio Weights: ", optimal_weights)

ValueError: Mismatch in dimensions of weights and covariance matrix